In [ ]:
import numpy as np
import os
import pandas
import theano

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import matplotlib.animation as animation

#%matplotlib inline

In [ ]:
dataset_file_path = './Data_RNN/201601.txt'
df = pandas.read_csv(dataset_file_path, delim_whitespace=True, header=None, usecols=[6])

# dataframe to dataset
dataset = df.values
dataset = dataset.astype('float32')
dataset = np.log10(dataset)

# normalize
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)

In [ ]:
plt.plot(dataset)   
plt.show()

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        dataX.append(dataset[i:(i+look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# 훈련은 0~10으로 테스트는 11로

look_back = 24

trainX, trainY = create_dataset(dataset, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

testX, testY = create_dataset(dataset, look_back)
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
%%time
theano.config.compute_test_value = "ignore"

# create and fit the LSTM network
batch_size = 1
model = Sequential()

for i in range(2):
    model.add(LSTM(64, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
    model.add(Dropout(0.3))
model.add(LSTM(64, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
for epoch_idx in range(10):

    model.fit(trainX, trainY, nb_epoch=1, batch_size=1, verbose=2, shuffle=False)
    model.reset_states()

In [ ]:
testScore = model.evaluate(testX, testY, batch_size=batch_size, verbose=0)
print('Test Score: ', testScore)

model.reset_states()

In [ ]:
total_length = 288 - look_back

for curr_idx in range(total_length):

    look_hist = curr_idx
    look_ahead = total_length - curr_idx

    predictions = np.zeros((total_length,1))
    testPredict = [np.vstack([testX[0][:]])]

    for i in range(look_hist):
        prediction = model.predict(np.array([testPredict[-1]]), batch_size=batch_size)
        predictions[i] = prediction
        testPredict.append(np.vstack([testX[i+1][:]]))
        
    for i in range(look_ahead):    
        prediction = model.predict(np.array([testPredict[-1]]), batch_size=batch_size)
        predictions[i + look_hist] = prediction
        testPredict.append(np.vstack([testPredict[-1][1:],prediction]))
    
    model.reset_states()
    

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate

fig = plt.figure(figsize=(12,5))    
ax = plt.axes(xlim=(0, 287), ylim=(0, 1))
plt.xlabel('time')
plt.ylabel('flux')

plotcols = [2], ["black","blue"]

predictions = dataset

lines = []

for i in range(2):
    line = ax.plot([], [], lw=2, color = plotcols[i])[0]
    lines.append(line)

# initialization function: plot the background of each frame
def init():
    
    for line in lines:
        line.set_data([],[])
    return lines

def animate(update_idx):

    look_real = update_idx
    look_ahead = 288-update_idx

    predictions = np.zeros((look_real + look_ahead, 1))

    for i in range(look_real):
        predictions[i] = dataset[i]

    for i in range(look_ahead):
        predictions[look_real + i] = 0

    x = np.linspace(0, 287, 288)

    y0 = dataset    
    y1 = predictions
    
    lines[0].set_data(x, y0)
    lines[1].set_data(x, y1)
    
    return lines
        
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=288, interval=20, blit=True)

#anim.save('exAnimation.gif', writer='imagemagick', fps=30, dpi=100)

plt.show()